In [1]:
import gradio as gr
import openai
import requests
from datetime import datetime
import os
import sys
from config import API_KEYS, DB_CONFIG, SYSTEM_PROMPT

Available names in config: ['API_KEYS', 'DB_CONFIG', 'SYSTEM_PROMPT', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'config']


In [4]:
class MedicalConsultation:
    def __init__(self):
        # OpenAI 클라이언트 초기화
        openai.api_key = API_KEYS["OPENAI"]
        self.pubmed_key = API_KEYS["PUBMED"]

    def search_pubmed(self, query: str, max_results: int = 3):
        """PubMed에서 관련 논문 검색"""
        base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils"
        
        # 검색 요청
        search_params = {
            "db": "pubmed",
            "term": query,
            "retmax": max_results,
            "api_key": self.pubmed_key,
            "retmode": "json"
        }
        
        try:
            # 논문 ID 검색
            search_response = requests.get(f"{base_url}/esearch.fcgi", params=search_params)
            search_data = search_response.json()
            
            if 'esearchresult' in search_data and 'idlist' in search_data['esearchresult']:
                paper_ids = search_data['esearchresult']['idlist']
                
                # 논문 상세 정보 요청
                summary_params = {
                    "db": "pubmed",
                    "id": ",".join(paper_ids),
                    "api_key": self.pubmed_key,
                    "retmode": "json"
                }
                
                summary_response = requests.get(f"{base_url}/esummary.fcgi", params=summary_params)
                summary_data = summary_response.json()
                
                papers = []
                for paper_id in paper_ids:
                    if paper_id in summary_data['result']:
                        paper = summary_data['result'][paper_id]
                        papers.append({
                            "title": paper.get('title', ''),
                            "authors": [author.get('name', '') for author in paper.get('authors', [])[:3]],
                            "journal": paper.get('source', ''),
                            "publication_date": paper.get('pubdate', ''),
                            "url": f"https://pubmed.ncbi.nlm.nih.gov/{paper_id}/"
                        })
                return papers
            
            return []
            
        except Exception as e:
            print(f"PubMed API 오류: {str(e)}")
            return []

    def get_medical_response(self, query: str):
        """의학 상담 응답 생성"""
        if not query.strip():
            return "질문을 입력해주세요.", []

        try:
            # PubMed에서 관련 논문 검색
            relevant_papers = self.search_pubmed(query)
            
            # 논문 정보 포맷팅
            papers_context = "\n".join([
                f"논문: {paper['title']} ({paper['journal']}, {paper['publication_date']})"
                for paper in relevant_papers
            ])

            # ChatGPT API 호출
            chat_response = openai.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": f"질문: {query}\n\n참고할 수 있는 논문 정보:\n{papers_context}"}
                ],
                max_tokens=1000
            )
            
            return chat_response.choices[0].message.content, relevant_papers
            
        except Exception as e:
            print(f"응답 생성 오류: {str(e)}")
            return "죄송합니다. 응답을 생성하는 중에 오류가 발생했습니다.", []

def translate_to_english(self, korean_query: str) -> str:
    """한국어 질문을 영어로 번역"""
    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a translator. Translate the given Korean medical question to English for PubMed search. Keep medical terms accurate."},
                {"role": "user", "content": korean_query}
            ],
            max_tokens=200
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"번역 오류: {str(e)}")
        return korean_query  # 오류 발생시 원본 반환

def get_medical_response(self, query: str):
    """의학 상담 응답 생성"""
    if not query.strip():
        return "질문을 입력해주세요.", []

    try:
        # 한국어 질문을 영어로 번역
        english_query = self.translate_to_english(query)
        print(f"Translated query: {english_query}")  # 디버깅용
        
        # PubMed에서 관련 논문 검색 (영문 검색어 사용)
        relevant_papers = self.search_pubmed(english_query)
        
        # 논문 정보 포맷팅
        papers_context = "\n".join([
            f"논문: {paper['title']} ({paper['journal']}, {paper['publication_date']})"
            for paper in relevant_papers
        ])

        # ChatGPT API 호출 (한국어 질문 사용)
        chat_response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"""
질문: {query}

참고할 수 있는 논문 정보:
{papers_context}

영문 검색어로 사용된 키워드: {english_query}
"""}
            ],
            max_tokens=1000
        )
        
        return chat_response.choices[0].message.content, relevant_papers
        
    except Exception as e:
        print(f"응답 생성 오류: {str(e)}")
        return "죄송합니다. 응답을 생성하는 중에 오류가 발생했습니다.", []

def create_consultation_tab():
    """일반 상담 탭 인터페이스 생성"""
    try:
        consultation = MedicalConsultation()
    except Exception as e:
        print(f"초기화 오류: {str(e)}")
        return None

    with gr.Blocks() as demo:
        gr.Markdown("# 의학 정보 상담")
        
        with gr.Row():
            with gr.Column(scale=2):
                query_input = gr.Textbox(
                    label="질문을 입력하세요",
                    placeholder="예: 고혈압의 주요 증상은 무엇인가요?",
                    lines=3
                )
                chat_btn = gr.Button("질문하기")
            
            with gr.Column(scale=3):
                response_output = gr.Textbox(label="답변", lines=10)
                papers_output = gr.JSON(label="관련 논문")
        
        chat_btn.click(
            consultation.get_medical_response,
            inputs=[query_input],
            outputs=[response_output, papers_output]
        )
    
    
    return demo

In [5]:
if __name__ == "__main__":
    demo = create_consultation_tab()
    if demo:
        demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


응답 생성 오류: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
